## Module 9: Test granular access control using Athena query

Using Row- and Cell-level access using AWS Lake Formation. Extract the data from Feature Store using Athena query

![Granular Access using Lake Formation](../../images/m9_nb1_lakeformation_architecture.png "Granular Access using Lake Formation")

#### Imports 

In [12]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role
import sagemaker
import logging
import boto3
import pandas as pd
import time
import re
import os
import sys
from pyathena import connect
import pandas as pd
sys.path.append('..')
from utilities import Utils
from utilities import feature_store_helper

In [13]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Initialize default parameters

In [14]:
sagemaker_session = sagemaker.Session()
account_id = sagemaker_session.account_id()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()
s3_client = boto3.client('s3', region_name=region)
query_results= 'sagemaker-featurestore-workshop'
prefix = 'sagemaker-feature-store'

#### Initialize boto3 runtime

In [15]:
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = sagemaker.Session(boto_session=boto_session, 
                                          sagemaker_client=sagemaker_client, 
                                          sagemaker_featurestore_runtime_client=featurestore_runtime)

#### Retrieve the orders products and customers feature group names

In this notebook, we will be using the feature groups created in Module 1 of the workshop. In case you would like to use a different feature group, please uncomment and replace **ORDERS_FEATURE_GROUP_NAME**, **CUSTOMERS_FEATURE_GROUP_NAME**, **PRODUCTS_FEATURE_GROUP_NAME** with the name of your Feature Groups for example. 

In [18]:
# Retrieve FG names (when running previous modules)
#%store -r customers_feature_group_name
#%store -r orders_feature_group_name
#%store -r products_feature_group_name

orders_feature_group_name = 'fscw-orders-02-13-20-20'
customers_feature_group_name = 'fscw-customers-02-13-20-20'
products_feature_group_name = 'fscw-products-02-13-20-20'

logger.info(f'Customers feature group name = {customers_feature_group_name}')
logger.info(f'Products feature group name = {products_feature_group_name}')
logger.info(f'Orders feature group name = {orders_feature_group_name}')

customers_fg = FeatureGroup(name=customers_feature_group_name, sagemaker_session=feature_store_session)  
orders_fg = FeatureGroup(name=orders_feature_group_name, sagemaker_session=feature_store_session)
products_fg = FeatureGroup(name=products_feature_group_name, sagemaker_session=feature_store_session)

Customers feature group name = fscw-customers-02-13-20-20
Customers feature group name = fscw-customers-02-13-20-20
Products feature group name = fscw-products-02-13-20-20
Products feature group name = fscw-products-02-13-20-20
Orders feature group name = fscw-orders-02-13-20-20
Orders feature group name = fscw-orders-02-13-20-20


#### Build Athena query to retrieve records from features groups - `customers` `products` `orders`

In [19]:
customers_query = customers_fg.athena_query()
customers_table = customers_query.table_name

products_query = products_fg.athena_query()
products_table = products_query.table_name

orders_query = orders_fg.athena_query()
orders_table = orders_query.table_name

In [20]:
orders_query_string = f'SELECT * FROM "sagemaker_featurestore"."{orders_table}" '
customers_query_string = f'SELECT * FROM "sagemaker_featurestore"."{customers_table}" '
products_query_string = f'SELECT * FROM "sagemaker_featurestore"."{products_table}" '

%store orders_query_string
orders_query_string

%store customers_query_string
customers_query_string

%store products_query_string
products_query_string

Stored 'orders_query_string' (str)
Stored 'customers_query_string' (str)
Stored 'products_query_string' (str)


'SELECT * FROM "sagemaker_featurestore"."fscw-products-02-13-20-20-1644783659" '

In [21]:
output_location = f's3://{default_bucket}/athena-results/'
print(f'Athena query output location: \n{output_location}')

Athena query output location: 
s3://sagemaker-us-east-1-227246955871/athena-results/


### Scenario 1: Row Level Access using Lake Formation using `orders`

#### Lakeformation applies row level access control and the sql query returns data only for one customer Id.

##### Extract the data from Feature Store using Athena query using SageMaker SDK

In [22]:
orders_query.run(query_string=orders_query_string, output_location=output_location)
orders_query.wait()
orders_df = orders_query.as_dataframe()
orders_df.head()

,order_id,customer_id,product_id,purchase_amount,is_reordered,event_time,n_days_since_last_purchase,write_time,api_invocation_time,is_deleted
0,O13396,C7782,P12059,0.172871,1,2022-02-13T20:18:07.415Z,0.116279,2022-02-13 20:27:26.832,2022-02-13 20:22:47.000,False
1,O15608,C7782,P9901,0.282871,1,2022-02-13T20:18:07.540Z,0.170543,2022-02-13 20:27:27.962,2022-02-13 20:23:36.000,False
2,O55559,C7782,P2748,0.694653,0,2022-02-13T20:18:09.879Z,0.434109,2022-02-13 20:27:27.613,2022-02-13 20:24:33.000,False
3,O48986,C7782,P8904,0.487723,0,2022-02-13T20:18:09.503Z,0.717054,2022-02-13 20:27:27.083,2022-02-13 20:24:23.000,False
4,O26086,C7782,P9474,0.337129,0,2022-02-13T20:18:08.141Z,0.077519,2022-02-13 20:27:28.419,2022-02-13 20:22:52.000,False


### Scenario 2: Column Level Access using Lake Formation using `Products`
#### Lakeformation applies column level access control and excludes multiple columns from the resultset.

##### Extract the data from Feature Store using Athena query using SageMaker SDK

In [ ]:
products_query.run(query_string=products_query_string, output_location=output_location)
products_query.wait()
products_df = products_query.as_dataframe()
products_df.head()

### Scenario 3: Cell Level Access using Lake Formation using `Customers`
#### Lakeformation applies both row and column level ( also known as cell level) access control. The athena query result exclude two columns and return rows for a specific customer Id.

##### Extract the data from Feature Store using Athena query using SageMaker SDK

In [ ]:
customers_query.run(query_string=customers_query_string, output_location=output_location)
customers_query.wait()
customers_df = customers_query.as_dataframe()
customers_df.head()

### For clean up activities, please refer to `m9_nb2_row_cell_level_access_clean_up.ipynb`